<a href="https://colab.research.google.com/github/anismehmahi/propaganda_detection/blob/main/chatGPT_4_3-5_turbo_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import asyncio
import aiohttp

headers = {
    "Content-Type": "application/json",
    "Authorization": f"OpenAI Key Here"
}

In [ ]:
import os
import nltk
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# import shutil

# shutil.rmtree("/content/split-articles")


## paper_prompt_base : papers' prompt

In [ ]:
paper_prompt_base = f"""
You are a multi-label text classifier indetifying 14 propaganda techniques within news paper articles. These are the 14 propaganda techniques you classify with definitions and examples:
Loaded_Language - Uses specific phrases and words that carry strong emotional impact to affect the audience, e.g. 'a lone lawmaker’s childish shouting.'
Name_Calling,Labeling - Gives a label to the object of the propaganda campaign as either the audience hates or loves, e.g. 'Bush the Lesser.'
Repetition -  Repeats the message over and over in the article so that the audience will accept it, e.g. 'Our great leader is the epitome of wisdom. Their decisions are always wise and just.'
Exaggeration,Minimisation - Either representing something in an excessive manner or making something seem less important than it actually is, e.g. 'I was not fighting with her; we were just playing.'
Appeal_to_fear-prejudice - Builds support for an idea by instilling anxiety and/or panic in the audience towards an alternative, e.g. 'stop those refugees; they are terrorists.'
Flag-Waving; Playing on strong national feeling (or with respect to a group, e.g., race, gender, political preference) to justify or promote an action or idea, e.g. 'entering this war will make us have a better future in our country.'
Causal_Oversimplification -  Assumes a single reason for an issue when there are multiple causes, e.g. 'If France had not declared war on Germany, World War II would have never happened.'
Appeal_to_Authority - Supposes that a claim is true because a valid authority or expert on the issue supports it, 'The World Health Organisation stated, the new medicine is the most effective treatment for the disease.'
Slogans - A brief and striking phrase that contains labeling and stereotyping, e.g.  “Make America great again!”
Thought-terminating_Cliches -  Words or phrases that discourage critical thought and useful discussion about a given topic, e.g. “it is what it is”
Whataboutism,Straw_Men,Red_Herring - Attempts to discredit an opponent’s position by charging them with hypocrisy without directly disproving their argument, e.g. 'They want to preserve the FBI’s reputation.'
Black-and-White_Fallacy -  Gives two alternative options as the only possibilities, when actually more options exist, e.g. 'You must be a Republican or Democrat'
Bandwagon,Reductio_ad_hitlerum - Justify actions or ideas because everyone else is doing it, or reject them because it's favored by groups despised by the target audience, e.g. “Would you vote for Clinton as president? 57% say yes.”
Doubt - Questioning the credibility of someone or something, e.g. 'Is he ready to be the Mayor?'

For the given article please state which of the 14 propaganda techniques are present. If no propaganda technique was identified return "no propaganda detected". An example output would list the propaganda techniques with each technique in a new line, e.g.:
Loaded_Language
Thought-terminating_Cliches
Repetition
Here is the article:

"""



##paper_prompt_COT : chain-of-thought papers' prompt

In [ ]:

paper_prompt_COT = f"""
You are a multi-label text classifier indetifying 14 propaganda techniques within news paper articles. These are the 14 propaganda techniques you classify with definitions and examples:
Loaded_Language - Uses specific phrases and words that carry strong emotional impact to affect the audience, e.g. 'a lone lawmaker’s childish shouting.'
Name_Calling,Labeling - Gives a label to the object of the propaganda campaign as either the audience hates or loves, e.g. 'Bush the Lesser.'
Repetition -  Repeats the message over and over in the article so that the audience will accept it, e.g. 'Our great leader is the epitome of wisdom. Their decisions are always wise and just.'
Exaggeration,Minimisation - Either representing something in an excessive manner or making something seem less important than it actually is, e.g. 'I was not fighting with her; we were just playing.'
Appeal_to_fear-prejudice - Builds support for an idea by instilling anxiety and/or panic in the audience towards an alternative, e.g. 'stop those refugees; they are terrorists.'
Flag-Waving; Playing on strong national feeling (or with respect to a group, e.g., race, gender, political preference) to justify or promote an action or idea, e.g. 'entering this war will make us have a better future in our country.'
Causal_Oversimplification -  Assumes a single reason for an issue when there are multiple causes, e.g. 'If France had not declared war on Germany, World War II would have never happened.'
Appeal_to_Authority - Supposes that a claim is true because a valid authority or expert on the issue supports it, 'The World Health Organisation stated, the new medicine is the most effective treatment for the disease.'
Slogans - A brief and striking phrase that contains labeling and stereotyping, e.g.  “Make America great again!”
Thought-terminating_Cliches -  Words or phrases that discourage critical thought and useful discussion about a given topic, e.g. “it is what it is”
Whataboutism,Straw_Men,Red_Herring - Attempts to discredit an opponent’s position by charging them with hypocrisy without directly disproving their argument, e.g. 'They want to preserve the FBI’s reputation.'
Black-and-White_Fallacy -  Gives two alternative options as the only possibilities, when actually more options exist, e.g. 'You must be a Republican or Democrat'
Bandwagon,Reductio_ad_hitlerum - Justify actions or ideas because everyone else is doing it, or reject them because it's favored by groups despised by the target audience, e.g. “Would you vote for Clinton as president? 57% say yes.”
Doubt - Questioning the credibility of someone or something, e.g. 'Is he ready to be the Mayor?'

For the given article please state which of the 14 propaganda techniques are present and give an explanation to why the technique is present in the article. If no propaganda technique was identified return "no propaganda detected". An example output would list the propaganda techniques with each technique in a new line, e.g.:
Loaded_Language - Your explanation why this technique is present in the article.
Thought-terminating_Cliches - Your explanation why this technique is present in the article.
Repetition - Your explanation why this technique is present in the article.
Here is the article:

"""

## guidlines_prompt : give sentences + text and gpt classify those sentences

In [ ]:
guidlines_prompt = f'''
You are expert in propaganda detection, here are guidelines for annotating a text to determine whether it contains propaganda or not or can not be classified :

  Exposes a rational argument?
   - Yes:
      - do you need more external information to judge ?
        - Yes: IDK
        - No:
          - Adds irrelevant data or changes the argument?
            - Yes: Propaganda (Whataboutism)
            - No:
                - Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered ?
                  - Yes: Propaganda (Appeal to authority)
                  - No:
                      - Starts with obvious but too abstract things?
                        - Yes: Propaganda
                        - No: Not propaganda
   - No:
      - Object of the sentence with respect to the main topic is:
         1. The very same topic:
            - Uses emotional words?
               - Yes: to prove an argument (appeal-to-fear, repetition, doubt) or without concrete argument (loaded language, flag-waving) then : Propaganda
               - No: Not propaganda
         2. A person with a different position:
            - Target being attacked/diminished?
               - Yes: Propaganda (name calling / labeling)
               - No: Not propaganda
         3. A third figure:
            - Tries to persuade?
               - Yes: to approve (Propaganda reductio ad hitlerum ) or to disapprove (Propaganda bandwagon)
               - No: Not propaganda

given the guidlines above, here is a text :

'''



# guidlines_prompt = f'''
# You are expert in propaganda detection, here are guidelines for annotating a text to determine whether it contains propaganda or not or can not be classified :

#   Exposes a rational argument?
#    - Yes:
#       - do you need more external information to judge ?
#         - Yes: IDK
#         - No:
#           - Adds irrelevant data or changes the argument?
#             - Yes: Propaganda (Whataboutism)
#             - No:
#                 - Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered ?
#                   - Yes: Propaganda (Appeal to authority)
#                   - No:
#                       - Starts with obvious but too abstract things?
#                         - Yes: Propaganda
#                         - No: Not propaganda
#                 - Trying to reduce the problem to only one cause ? : Propaganda Causal Oversimplification
#    - No:
#       - Object of the sentence with respect to the main topic is:
#          1. The very same topic:
#             - Uses emotional words?
#                - Yes: to prove an argument (appeal-to-fear, repetition, doubt) or without concrete argument (loaded language, flag-waving) then : Propaganda
#                - No: Not propaganda
#          2. A person with a different position:
#             - Target being attacked/diminished?
#                - Yes: Propaganda (name calling / labeling)
#                - No: Not propaganda
#          3. A third figure:
#             - Tries to persuade?
#                - Yes: to approve (Propaganda reductio ad hitlerum ) or to disapprove (Propaganda bandwagon)
#                - No: Not propaganda

# given the guidlines above, here is a text :
# {text}

# - what techniques are used in these sentences and give the path you used from the guidlines:
# {bullet_sentences}

# '''



## guidlines_prompt_2 : guidlines + no sentences + text and gpt extracts sentences and their techniques

In [ ]:


guidlines_prompt_2 = f'''
You are expert in propaganda detection, here are guidelines for annotating a text to determine whether it contains propaganda or not or can not be classified :

  Exposes a rational argument?
   - Yes:
      - do you need more external information to judge ?
        - Yes: IDK
        - No:
          - Adds irrelevant data or changes the argument?
            - Yes: Propaganda (Whataboutism)
            - No:
                - Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered ?
                  - Yes: Propaganda (Appeal to authority)
                  - No:
                      - Starts with obvious but too abstract things?
                        - Yes: Propaganda
                        - No: Not propaganda
   - No:
      - Object of the sentence with respect to the main topic is:
         1. The very same topic:
            - Uses emotional words?
               - Yes: to prove an argument (appeal-to-fear, repetition, doubt) or without concrete argument (loaded language, flag-waving) then : Propaganda
               - No: Not propaganda
         2. A person with a different position:
            - Target being attacked/diminished?
               - Yes: Propaganda (name calling / labeling)
               - No: Not propaganda
         3. A third figure:
            - Tries to persuade?
               - Yes: to approve (Propaganda reductio ad hitlerum ) or to disapprove (Propaganda bandwagon)
               - No: Not propaganda

given the guidlines above, here is a text :
{text}

- what techniques are used in the text and give the path you used from the guidlines.
'''


## prompt_definitions : definitions + give sentences + text and gpt classify those sentences

In [ ]:
prompt_definitions = f'''

Here are some definitions related to propaganda and how to detecte the technique used for it, please read them carrefuly.

1.Whataboutism
A technique that attempts to discredit an opponent's position by charging them with hypocrisy without directly disproving their argument.
Example 1: a nation deflects criticism of its recent human rights violations by
pointing to the history of slavery in the United States.
Example 2: "Qatar spending profusely on Neymar, not fighting terrorism"

2.Causal Oversimplification
Assuming a single cause or reason when there are actually multiple causes for an issue.
It includes transferring  blame to one  person or  group of people without investigating  the complexities of the issue
Example 1: "President Trump has been in office for a month and gas prices have been skyrocketing. The rise in gas prices is because of President Trump."
Example 2: The reason New Orleans was hit so hard with the hurricane was
because of all the immoral people who live there.
Explanation: This was an actual argument seen in the months that followed
hurricane Katrina. Ignoring the validity of the claims being made,
the arguer is blaming a natural disaster on a group of people. Example 3: if France had not have declared war on Germany then world war two
would have never happened.

3. Appeal to authority
Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered.
Example: Richard Dawkins, an evolutionary biologist and perhaps the foremost expert in the field, says that evolution is true. Therefore, it's true. Explanation: Richard Dawkins certainly knows about evolution, and he can confidently tell us that it is true, but that doesn't make it true. What makes it true is the preponderance of evidence for the theory. Example 2: "According to Serena Williams, our foreign policy is the best on Earth. So we are in the right direction." Details: since there is a chance that any authority can be wrong, it is reasonable to defer to an authority to support a claim, but the authority should not be the only justification to accept the claim, otherwise the Appeal-to-Authority fallacy is committed.

4. Black-and-white Fallacy, Dictatorship
Presenting two alternative options as the only possibilities, when in fact more possibilities exist. As an the extreme case, tell the audience exactly what actions to take, eliminating any other possible choices (Dictatorship).
Example 1: You must be a Republican or Democrat. You are not a Democrat. Therefore, you must be a Republican Example 2: I thought you were a good person, but you weren't at church today. Explanation: The assumption here is that if one doesn't attend church, one must be bad. Of course, good people exist who don't go to church, and good church-going people could have had a really good reason not to be in church. Example 3: There is no alternative to war

5. Name calling or labeling
Labeling the object of the propaganda campaign as either something the target audience fears, hates, finds undesirable or loves, praises.
Examples: "Republican congressweasels", "Bush the Lesser" (note that lesser does not refer to "the second", but it is pejorative)

6. Loaded Language
Using specific words and phrases with strong emotional implications (either positive or negative) to influence an audience.
Example 1: "[...] a lone lawmaker's childish shouting. ". Example 2: "how stupid and petty things have become in Washington"

7. Exaggeration or Minimisation
Either representing something in an excessive manner: making things larger, better, worse (e.g., "the best of the best", "quality guaranteed") or making something seem less important or smaller than it really is (e.g., saying that an insult was just a joke).
Example 1: "Democrats bolted as soon as Trump's speech ended in an apparent effort to signal they can't even stomach being in the same room as the president " Example 2: "We're going to have unbelievable intelligence" Example 3: I was not fighting with her; we were just playing.

8. Flag-waving
Playing on strong national feeling (or to any group; e.g., race, gender, political preference) to justify or promote an action or idea
Example 1: "patriotism mean no questions" (this is also a slogan) Example 2: "entering this war will make us have a better future in our country."

9. Doubt
Questioning the credibility of someone or something.
Example: A candidate talks about his opponent and says: Is he ready to be the Mayor?

10. Appeal to fear/prejudice
Seeking to build support for an idea by instilling anxiety and/or panic in the population towards an alternative. In some cases the support is built based on preconceived judgements.
Example 1: "either we go to war or we will perish" (this is also a Black and White fallacy)) Example 2: "we must stop those refugees as they are terrorists"

11. Repetition
Repeating the same message over and over again so that the audience will eventually accept it.
Example : It won’t take away Iran’s enriched uranium, ... It won’t destroy Iran’s nuclear installations ... An improved deal won’t destroy Iran’s centrifuges.


12. Reductio ad hitlerum
Persuading an audience to disapprove an action or idea by suggesting that the idea is popular with
groups hated in contempt by the target audience. It can refer to any person or concept with a
negative connotation.
Example 1: "Do you know who else was doing that ? Hitler!"
Example 2: "Only one kind of person can think in that way: a communist."


given the definitions above, here is a text :
{text}

- what techniques are used in each of these sentences :
{bullet_sentences}

'''

## prompt_definitions_2 : definitions + no sentences + text and gpt extracts sentences and their techniques

In [ ]:
prompt_definitions_2 = f'''

Here are some definitions related to propaganda and how to detecte the technique used for it, please read them carrefuly.

1.Whataboutism
A technique that attempts to discredit an opponent's position by charging them with hypocrisy without directly disproving their argument.
Example 1: a nation deflects criticism of its recent human rights violations by
pointing to the history of slavery in the United States.
Example 2: "Qatar spending profusely on Neymar, not fighting terrorism"

2.Causal Oversimplification
Assuming a single cause or reason when there are actually multiple causes for an issue.
It includes transferring  blame to one  person or  group of people without investigating  the complexities of the issue
Example 1: "President Trump has been in office for a month and gas prices have been skyrocketing. The rise in gas prices is because of President Trump."
Example 2: The reason New Orleans was hit so hard with the hurricane was
because of all the immoral people who live there.
Explanation: This was an actual argument seen in the months that followed
hurricane Katrina. Ignoring the validity of the claims being made,
the arguer is blaming a natural disaster on a group of people. Example 3: if France had not have declared war on Germany then world war two
would have never happened.

3. Appeal to authority
Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered.
Example: Richard Dawkins, an evolutionary biologist and perhaps the foremost expert in the field, says that evolution is true. Therefore, it's true. Explanation: Richard Dawkins certainly knows about evolution, and he can confidently tell us that it is true, but that doesn't make it true. What makes it true is the preponderance of evidence for the theory. Example 2: "According to Serena Williams, our foreign policy is the best on Earth. So we are in the right direction." Details: since there is a chance that any authority can be wrong, it is reasonable to defer to an authority to support a claim, but the authority should not be the only justification to accept the claim, otherwise the Appeal-to-Authority fallacy is committed.

4. Black-and-white Fallacy, Dictatorship
Presenting two alternative options as the only possibilities, when in fact more possibilities exist. As an the extreme case, tell the audience exactly what actions to take, eliminating any other possible choices (Dictatorship).
Example 1: You must be a Republican or Democrat. You are not a Democrat. Therefore, you must be a Republican Example 2: I thought you were a good person, but you weren't at church today. Explanation: The assumption here is that if one doesn't attend church, one must be bad. Of course, good people exist who don't go to church, and good church-going people could have had a really good reason not to be in church. Example 3: There is no alternative to war

5. Name calling or labeling
Labeling the object of the propaganda campaign as either something the target audience fears, hates, finds undesirable or loves, praises.
Examples: "Republican congressweasels", "Bush the Lesser" (note that lesser does not refer to "the second", but it is pejorative)

6. Loaded Language
Using specific words and phrases with strong emotional implications (either positive or negative) to influence an audience.
Example 1: "[...] a lone lawmaker's childish shouting. ". Example 2: "how stupid and petty things have become in Washington"

7. Exaggeration or Minimisation
Either representing something in an excessive manner: making things larger, better, worse (e.g., "the best of the best", "quality guaranteed") or making something seem less important or smaller than it really is (e.g., saying that an insult was just a joke).
Example 1: "Democrats bolted as soon as Trump's speech ended in an apparent effort to signal they can't even stomach being in the same room as the president " Example 2: "We're going to have unbelievable intelligence" Example 3: I was not fighting with her; we were just playing.

8. Flag-waving
Playing on strong national feeling (or to any group; e.g., race, gender, political preference) to justify or promote an action or idea
Example 1: "patriotism mean no questions" (this is also a slogan) Example 2: "entering this war will make us have a better future in our country."

9. Doubt
Questioning the credibility of someone or something.
Example: A candidate talks about his opponent and says: Is he ready to be the Mayor?

10. Appeal to fear/prejudice
Seeking to build support for an idea by instilling anxiety and/or panic in the population towards an alternative. In some cases the support is built based on preconceived judgements.
Example 1: "either we go to war or we will perish" (this is also a Black and White fallacy)) Example 2: "we must stop those refugees as they are terrorists"

11. Slogans
A brief and striking phrase that may include labeling and stereotyping. Slogans tend to act as emotional appeals.
Example 1: "The more women at war . . . the sooner we win." Example 2: "Make America great again!"

12. Bandwagon
Attempting to persuade the target audience to join in and take the course of action because
"everyone else is taking the same action".
Example 1: Would you vote for Clinton as president? 57% say yes Example 2: 90% of citizens support
our initiative. You should.

13. Reductio ad hitlerum
Persuading an audience to disapprove an action or idea by suggesting that the idea is popular with
groups hated in contempt by the target audience. It can refer to any person or concept with a
negative connotation.

Example 1: "Do you know who else was doing that ? Hitler!"
Example 2: "Only one kind of person can think in that way: a communist."

14. Repetition
Repeating the same message over and over again so that the audience will eventually accept it.
Example : It won’t take away Iran’s enriched uranium, ... It won’t destroy Iran’s nuclear installations ... An improved deal won’t destroy Iran’s centrifuges.

given the definitions above, here is a text :
{text}

- what techniques are used in the given text, extract sentences where those techniques appeared ?

'''

## Split long articles (10000 TPM)

In [ ]:
# here we should split long text and get their labels too and then maybe we can pass them to gpt 4 and do chain of thoughts
# this function is used to divide test articles into 1500 tokens and get to each part the techniques used in it. this is all to get rid of that 10000 TPM caused in GPT-4

paths_in = ['/content/test-articles/',]
paths_out = ['/content/split-articles/test-articles/', ]

def write_to_file(text, file_name, labels, label_file_name):
    with open(file_name, 'w', encoding="utf-8") as f:
        f.write(text)

    with open(label_file_name, 'w', encoding="utf-8") as f:
        labels = list(set(labels))
        for label in labels:
            f.write(f'{label}\n')

for path_in, path_out in zip(paths_in, paths_out):
    files = os.listdir(path_in)
    files.sort()
    for file_name in files:
        chunk_size = 1000  # adjust this value as needed
        # print('dddd', len(files))
        with open(path_in+file_name, 'r') as f:
            text = f.read()
        if path_in.endswith('test-articles/'):
            label_df = pd.read_csv(path_in.replace('test-articles', 'test-labels-task-flc-tc') + file_name.split(".")[0]+".task-flc-tc.labels", sep='\t', header=None, names=['filename', 'label', 'start', 'end'])
        else:
            label_df = pd.read_csv(path_in.replace('dev-articles', 'dev-labels-task-flc-tc') + file_name.split(".")[0]+".task-flc-tc.labels", sep='\t', header=None, names=['filename', 'label', 'start', 'end'])

        # split the text into sentences
        sentences = nltk.sent_tokenize(text)

        subfile_index = 1
        subfile_sentences = []
        upper_boundary = 0
        lower_boundary = 0
        for sentence in sentences:
            sentence_length = len(sentence)
            if upper_boundary + sentence_length + 1 > chunk_size:  # +1 for the space between sentences
                # write the current chunk to a file
                subfile_text = ' '.join(subfile_sentences)
                if subfile_index == 1:
                    labels = label_df[label_df['end'] <= upper_boundary].label.values
                else:
                    labels = label_df[(label_df['start'] > lower_boundary)&(label_df['end'] <= upper_boundary)].label.values
                # print("subfile_index:", subfile_index, "boundaries:", lower_boundary, upper_boundary, "chunksize:", chunk_size)
                if path_out.endswith('test-articles/'):
                    # print("dddddddddd")
                    write_to_file(subfile_text, f'{path_out}{file_name.split(".")[0]}_{subfile_index}.txt', labels , f'{path_out.replace("test-articles", "test-labels")}{file_name.split(".")[0]}_{subfile_index}.txt')
                else:
                    write_to_file(subfile_text, f'{path_out}{file_name.split(".")[0]}_{subfile_index}.txt', labels , f'{path_out.replace("dev-articles", "dev-labels")}{file_name.split(".")[0]}_{subfile_index}.txt')
                # start a new chunk
                lower_boundary = upper_boundary
                subfile_sentences = [sentence]
                upper_boundary += sentence_length + 1
                subfile_index += 1
                chunk_size = upper_boundary + 1500
            else:
                # add the sentence to the current chunk
                subfile_sentences.append(sentence)
                upper_boundary += sentence_length + 1  # +1 for the space between sentences

        # don't forget the last chunk
        if subfile_sentences:
            subfile_text = ' '.join(subfile_sentences)
            labels = label_df[(label_df['end'] > lower_boundary)].label.values
            #print("subfile_index:", subfile_index, "boundaries:", lower_boundary, upper_boundary, "chunksize:", chunk_size)
            if path_out.endswith('test-articles/'):
                write_to_file(subfile_text, f'{path_out}{file_name.split(".")[0]}_{subfile_index}.txt', labels , f'{path_out.replace("test-articles", "test-labels")}{file_name.split(".")[0]}_{subfile_index}.txt')
            else:
                write_to_file(subfile_text, f'{path_out}{file_name.split(".")[0]}_{subfile_index}.txt', labels , f'{path_out.replace("dev-articles", "dev-labels")}{file_name.split(".")[0]}_{subfile_index}.txt')

## Predict

In [ ]:
import os
# prepare a list with all the prompt + text to give them to gpt
# Folder containing the articles
articles_folder = '/content/dev-articles/'
paper_prompt = paper_prompt_base
files_path = []
# Initialize an empty list to store prompts
prompts = []
files_name=sorted(os.listdir(articles_folder))
# Iterate through the files in the articles folder
for filename in files_name:
    if filename.endswith(".txt"):
        # Read the content of the article
        print(filename)
        files_path.append(filename)
        with open(os.path.join(articles_folder, filename), 'r', encoding='utf-8') as article_file:
            article_text = article_file.read()

        # Combine the base prompt with the article text
        combined_prompt = paper_prompt+article_text

        # Append the combined prompt to the list
        prompts.append(combined_prompt)


In [ ]:
len(prompts)

75

In [ ]:
!unzip "/content/sprenkamp LLM_propaganda_detection main data-propaganda_text_classification_full-articles_dev-articles.zip" -d "/content/dev-articles/"


In [ ]:
!unzip "/content/sprenkamp LLM_propaganda_detection main data-propaganda_text_classification_full-articles_dev-labels.zip" -d "/content/dev-labels/"


In [ ]:
# import os
# import glob

# # List all files in the current directory that start with 'article'
# files = glob.glob('article*')

# # Iterate over the list of filepaths & remove each file
# for file in files:
#     try:
#         os.remove(file)
#         print(f"Deleted {file}")
#     except OSError as e:
#         print(f"Error: {file} : {e.strerror}")


In [ ]:
partial_prompts = prompts

In [ ]:
# prblem with gpt-4 is TPM (token per minute) and some times prompt + article > 10K so impossible
import asyncio
import aiohttp
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)



completion_list = []
class ProgressLog:
    def __init__(self, total):
        self.total = total
        self.done = 0

    def increment(self):
        self.done = self.done + 1

    def __repr__(self):
        return f"Done runs {self.done}/{self.total}."

@retry(wait=wait_random_exponential(min=60, max=80), stop=stop_after_attempt(2), before_sleep=print, retry_error_callback=lambda _: None)
async def get_completion(content, session, semaphore, progress_log):
    async with semaphore:
        async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json={
            "model": "gpt-4-turbo-preview",
            "messages": [{"role": "user", "content": content}],
            "temperature": 0
        }) as resp:

            response_json = await resp.json()

            progress_log.increment()
            print(response_json)

            return response_json["choices"][0]['message']["content"]

async def get_completion_list(content_list, max_parallel_calls, timeout):
    semaphore = asyncio.Semaphore(value=max_parallel_calls)
    progress_log = ProgressLog(len(content_list))

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(timeout)) as session:
        return await asyncio.gather(*[get_completion(content, session, semaphore, progress_log) for content in content_list])


start_time = time.perf_counter()


completion_list = await get_completion_list(partial_prompts,1, 40)
print("Time elapsed: ", time.perf_counter() - start_time, "seconds.")
# print(completion_list[0])

{'id': 'chatcmpl-8oZp1ButtbfBAqxKI0cflVilVksbV', 'object': 'chat.completion', 'created': 1707064791, 'model': 'gpt-4-0125-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'no propaganda detected'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1051, 'completion_tokens': 3, 'total_tokens': 1054}, 'system_fingerprint': 'fp_f084bcfc79'}
{'id': 'chatcmpl-8oZp2Wm4uAZQ1Ud5rpMpmEG6eML5x', 'object': 'chat.completion', 'created': 1707064792, 'model': 'gpt-4-0125-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Loaded_Language\nAppeal_to_fear-prejudice\nCausal_Oversimplification\nDoubt'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1277, 'completion_tokens': 23, 'total_tokens': 1300}, 'system_fingerprint': 'fp_f084bcfc79'}
{'id': 'chatcmpl-8oZp5QdPO85L8zgPmV4M4OIA1jY0w', 'object': 'chat.completion', 'created': 1707064795, 'model': 'gpt-4-0125-preview', 'choices': [{'index': 0, 'mess

In [ ]:
completion_list

In [ ]:
# check None value in case failed callbacks
print(sum(x is None for x in completion_list))


0


In [ ]:
cc = ['no propaganda detected' if v is None else v for v in completion_list]


In [ ]:
import os

# Define the folder where you want to store the files
output_folder = 'dev-preds'

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List of elements you want to write to separate files
elements = cc

# Iterate through the list and write each element to a separate file
for file_nm, element in zip(files_path, elements):
    # Create a file with a name based on the index
    file_name = f'{file_nm}'
    file_path = os.path.join(output_folder, file_name)

    # Write the element to the file
    with open(file_path, 'w') as file:
        file.write(element)

print(f'All elements have been written to files in the {output_folder} folder.')


All elements have been written to files in the dev-preds folder.


In [ ]:
!zip -r /content/gpt4-dev-predictions.zip /content/dev-preds
from google.colab import files
files.download("/content/gpt4-dev-predictions.zip")


# Results

In [ ]:
import os
import json
import argparse
import yaml
from tqdm import tqdm
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer



mode = 'base'

def calculate_metrics():
    true_labels = []
    pred_labels = []

    val_label_path = "/content/dev-labels"
    pred_label_path = "/content/dev-preds"

    filenames = sorted([filename for filename in os.listdir(val_label_path) if filename.endswith('.txt')])
    for filename in filenames:
        true_label_file = []
        file_path = os.path.join(val_label_path, filename)
        if os.stat(file_path).st_size == 0:
            true_labels.append(true_label_file)
        else:
            with open(file_path, 'r', encoding="utf-8") as file:
                for line in file:
                    line = line.strip()
                    true_label_file.append(line)
            true_labels.append(true_label_file)
        pred_label_file = []
        allowed_labels = ['Appeal_to_Authority', 'Appeal_to_fear-prejudice', 'Bandwagon,Reductio_ad_hitlerum', 'Black-and-White_Fallacy', 'Causal_Oversimplification', 'Doubt', 'Exaggeration,Minimisation', 'Flag-Waving', 'Loaded_Language', 'Name_Calling,Labeling', 'Repetition', 'Slogans', 'Thought-terminating_Cliches', 'Whataboutism,Straw_Men,Red_Herring']
        with open(os.path.join(pred_label_path, filename), 'r', encoding="utf-8") as file:
            if os.stat(os.path.join(pred_label_path, filename)).st_size == 0:
                pred_labels.append(pred_label_file)
            else:
                if mode in ['base', 'noinstruction']:
                    for line in file:
                        line = line.strip()
                        if line in allowed_labels:
                            pred_label_file.append(line)
                elif mode == 'chain_of_thought':
                    for line in file:
                        line = line.strip()
                        label = line.split('-', 1)[0].strip()
                        if label in allowed_labels:
                            pred_label_file.append(label)
                pred_labels.append(pred_label_file)


    # Convert labels to binary representation using MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    true_labels_binary = mlb.fit_transform(true_labels)
    predicted_labels_binary = mlb.transform(pred_labels)

    # Define the output file path
    metric_file_path = os.path.join('/content/',  'metrics.txt')

    # Open the file in write mode
    with open(metric_file_path, "w") as file:
        # Accuracy
        accuracy = accuracy_score(true_labels_binary, predicted_labels_binary)
        file.write(f"Accuracy: {accuracy}\n")

        # Hamming Loss
        hamming_loss_value = hamming_loss(true_labels_binary, predicted_labels_binary)
        file.write(f"Hamming Loss: {hamming_loss_value}\n")

        # Precision, Recall, and F1-Score (Micro-Averaged)
        precision = precision_score(true_labels_binary, predicted_labels_binary, average='micro')
        recall = recall_score(true_labels_binary, predicted_labels_binary, average='micro')
        f1 = f1_score(true_labels_binary, predicted_labels_binary, average='micro')
        file.write(f"Micro-Averaged Precision: {precision}\n")
        file.write(f"Micro-Averaged Recall: {recall}\n")
        file.write(f"Micro-Averaged F1-Score: {f1}\n")

        # Subset Accuracy
        subset_accuracy = accuracy_score(true_labels_binary, predicted_labels_binary, normalize=True)
        file.write(f"Subset Accuracy: {subset_accuracy}\n")

        # Macro-Averaged Precision, Recall, and F1-Score
        macro_precision = precision_score(true_labels_binary, predicted_labels_binary, average='macro')
        macro_recall = recall_score(true_labels_binary, predicted_labels_binary, average='macro')
        macro_f1 = f1_score(true_labels_binary, predicted_labels_binary, average='macro')
        file.write(f"Macro-Averaged Precision: {macro_precision}\n")
        file.write(f"Macro-Averaged Recall: {macro_recall}\n")
        file.write(f"Macro-Averaged F1-Score: {macro_f1}\n")

        # F1-Score per class
        class_f1_scores = f1_score(true_labels_binary, predicted_labels_binary, average=None)
        file.write("F1-Score per Class:\n")
        for label, f1_score_class in zip(mlb.classes_, class_f1_scores):
            file.write(f"{label}: {f1_score_class}\n")

        # Precision per class
        class_precision_scores = precision_score(true_labels_binary, predicted_labels_binary, average=None, zero_division=0)
        file.write("Precision per Class:\n")
        for label, precision_score_class in zip(mlb.classes_, class_precision_scores):
            file.write(f"{label}: {precision_score_class}\n")
    print("Evaluation metrics written to the file:", metric_file_path)

calculate_metrics()


Evaluation metrics written to the file: /content/metrics.txt


In [ ]:
# import shutil

# shutil.rmtree('/content/dev-preds')
